In [58]:

import os,sys,inspect

current_dir = os.path.dirname(os.path.abspath(
        inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

from agents import *
from search import *

from scipy.spatial import distance
#from notebook import psource


In [59]:
current_dir
#parent_dir

'C:\\Users\\josel\\AppData\\Local\\Temp\\ipykernel_10404'

In [60]:
# Now you can import a module from the parent directory

#random.seed(1234)

In [61]:
collections
collections.Iterable = collections.abc.Iterable
collections.Sequence = collections.abc.Sequence

In [62]:
# Agent Class - Fire brigade Agent - Model
class ModelFireman(Agent):
    location = [0, 0]
    Name = 'ModelFireman'
    Agent.isHoldingWater = False

    def takeWater(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

    def estinguishFire(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Fire):
            return True
        return False

In [63]:
# model = {(x, y): None for x in range(6) for y in range(6)}

def programAgent(percept):
    location, things = percept
    for t in things:
        # if model[tuple(location)] not in['Visited']:
        if isinstance(t, Fire):
            model[tuple(location)] = 'Fire'
            print('There is Fire')
            return 'Estinguish Fire'
        elif isinstance(t, Water):
            model[tuple(location)] = 'Water'
            print('There is Water')
            return 'Take Water'
        model[tuple(location)] = t  # Update the model here

    if not isinstance(model[tuple(location)], (Water, Fire)):
        model[tuple(location)] = 'Visited'

    choice = random.choice(('L', 'R', 'D', 'U'))
    if choice !=0:
        if choice == 'L':
            pass
        elif choice == 'R':
            pass
        elif choice == 'D':
            pass
        elif choice == 'U':
            pass
    return 'Move' + choice

    return programAgent  # , model

In [64]:
########################################
# THING CLASSES
########################################
# Thing Class - Water
class Water(Thing):
    Name='Water'
    pass

# Thing Class - Fire


class Fire(Thing):
    Name='Fire'
    pass


class Wall(Obstacle):
    Name='Wall'
    pass


class Path(Thing):
    pass

In [65]:
class Forrest2D(GraphicEnvironment):
    def percept(self, agent):
        listThings = []
        for things in self.list_things_at(agent.location):
            if things != agent:
                listThings.append(things)
        """By default, agent perceives things within a default radius."""
        return agent.location, listThings

    def thing_classes(self):
        return [Fire, Water, ModelFireman]

    def execute_action(self, agent, action):
        if action == 'MoveR':
            if agent.location[0]>=0 and agent.location[0]< 5:
                agent.location[0]+=1
                agent.performance -= 1
                # print('R here we go')
        elif action == 'MoveL':
            if agent.location[0]>0 and agent.location[0]<=5:
                agent.location[0]-=1
                agent.performance -= 1
                # print('L here we go')
        elif action == 'MoveU':
            if agent.location[1]>=0 and agent.location[1]<5:
                agent.location[1]+=1
                agent.performance -= 1
                # print('U here we go')
        elif action == 'MoveD':
            if agent.location[1]>0 and agent.location[1]<=5:
                agent.location[1]-=1
                agent.performance -= 1
                # print('D here we go')
        elif action == 'Take Water':
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.takeWater(items[0]):
                    agent.performance += 100
                    self.delete_thing(items[0])
                    agent.isHoldingWater = True
                    print('New action to Take Water')
        elif action == 'Estinguish Fire':
            items = self.list_things_at(agent.location, tclass=Fire)
            if len(items) != 0:
                if agent.estinguishFire(items[0]) and agent.isHoldingWater:
                    agent.performance += 100
                    self.delete_thing(items[0])
                    agent.isHoldingWater = False
                    print('New action to Estinguish Fire')
                else:
                    agent.performance -= 100
                    print('Burning, no water')

            if model[tuple(agent.location)] == 'Visited':
                agent.performance -= 10

        print(model[tuple(agent.location)])
        print(agent.location)
        print(
            f'Fireman performance: {agent.performance} and is holding water: {agent.isHoldingWater}')

    def is_done(self):

        return False

In [66]:
width = 10
height = 10

In [67]:
model = {(x, y): None for x in range(width) for y in range(height)}
forrest = Forrest2D(width, height, color={'ModelFireman': (230, 115, 40), 'Water': (
    0, 200, 200), 'Fire': (200, 0, 0), 'Wall': (50, 50, 50), 'Path': (120, 0, 100)})

In [68]:
# Define the positions of obstacles.
obstacles = [(5, 4), (4, 4), (2, 4), (1, 4), (3, 4), (6, 4), (7, 4), (8, 4), (9, 4)]


def add_obstacles(self, obstacles):
    for each in obstacles:
        print(each)
        x, y = each
        self.add_thing(Wall(), (x, y))

In [69]:
fireman = ModelFireman(programAgent)
fire = Fire()
water = Water()
wall = Wall()

forrest.add_thing(fireman, [1, 1])
forrest.add_thing(water, [3, 4])
forrest.add_thing(fire, [3, 5])
add_obstacles(forrest, obstacles)

(5, 4)
(4, 4)
(2, 4)
(1, 4)
(3, 4)
(6, 4)
(7, 4)
(8, 4)
(9, 4)


In [70]:
forrest.run(500, delay=0.01)

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


In [71]:
print(f'Fireman performance: {fireman.performance} and is holding water: {fireman.isHoldingWater}')

Fireman performance: -226 and is holding water: False


In [72]:

###Search problem
def DicMapCreation(width,height,BlockPos):
    rows = height-1
    columns = width-1

    # Lista de posiciones bloqueadas
    GameMap = {}
    def is_inside_grid(x, y):
        return 0 < x <= columns and 0 < y <= rows

    # Generar las posibilidades de movimiento y sus costos, excluyendo las posiciones bloqueadas
    for x in range(1, columns + 1):
        for y in range(1, rows + 1):
            CurrPos = (x, y)
            
            # Verificar si la posición actual está bloqueada
            if CurrPos not in BlockPos:
                transitions = {}
                
                # Movimiento hacia arriba
                if is_inside_grid(x - 1,y) and (x - 1,y) not in BlockPos:
                    transitions[(x - 1,y)] = random.randint(1, 100)
                # Movimiento hacia abajo
                if is_inside_grid(x + 1,y) and (x + 1,y) not in BlockPos:
                    
                    transitions[(x + 1,y)] = random.randint(1, 100)
                # Movimiento hacia la izquierda
                if is_inside_grid(x,y - 1) and (x,y - 1) not in BlockPos:
                    transitions[(x,y - 1)] = random.randint(1, 100)
                # Movimiento hacia la derecha
                if is_inside_grid(x,y + 1) and (x,y + 1) not in BlockPos:
                    transitions[(x,y + 1)] = random.randint(1, 100)
                
                if transitions:
                    GameMap[CurrPos] = transitions

    # Prnint dictionary
    # for location, transitions in GameMap.items():
    #     print(f'From {location}: {transitions}')
    return GameMap
print(model)

{(0, 0): 'Visited', (0, 1): 'Visited', (0, 2): 'Visited', (0, 3): 'Visited', (0, 4): 'Visited', (0, 5): 'Visited', (0, 6): None, (0, 7): None, (0, 8): None, (0, 9): None, (1, 0): 'Visited', (1, 1): 'Visited', (1, 2): 'Visited', (1, 3): 'Visited', (1, 4): 'Visited', (1, 5): 'Visited', (1, 6): None, (1, 7): None, (1, 8): None, (1, 9): None, (2, 0): 'Visited', (2, 1): 'Visited', (2, 2): 'Visited', (2, 3): 'Visited', (2, 4): 'Visited', (2, 5): 'Visited', (2, 6): None, (2, 7): None, (2, 8): None, (2, 9): None, (3, 0): 'Visited', (3, 1): 'Visited', (3, 2): 'Visited', (3, 3): 'Visited', (3, 4): 'Visited', (3, 5): 'Visited', (3, 6): None, (3, 7): None, (3, 8): None, (3, 9): None, (4, 0): 'Visited', (4, 1): 'Visited', (4, 2): 'Visited', (4, 3): 'Visited', (4, 4): 'Visited', (4, 5): 'Visited', (4, 6): None, (4, 7): None, (4, 8): None, (4, 9): None, (5, 0): 'Visited', (5, 1): 'Visited', (5, 2): 'Visited', (5, 3): 'Visited', (5, 4): 'Visited', (5, 5): 'Visited', (5, 6): None, (5, 7): None, (5, 8):

In [73]:
class FirefightingProblem(Problem):
    def __init__(self, initial, goal=None, graph=None):
        super().__init__(initial, goal)
        self.graph = graph

    def actions(self, state):
        # Return possible movements (up, down, left, right) that do not lead to obstacles.
        x, y = state
        possible_actions = []

        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < len(self.graph) and 0 <= new_y < len(self.graph[0]) and self.graph[new_x][new_y] != 'Obstacle':
                possible_actions.append((new_x, new_y))

        return possible_actions

    def result(self, state, action):
        # Return the state after taking the given action.
        return action

    def goal_test(self, state):
        # Check if the state is at the exit point (goal).
        return state == self.goal

    def path_cost(self, cost_so_far, state1, action, state2):
        # Assign a cost of 1 per move (action).
        return cost_so_far + 1

    def h(self, node):
        # Estimate the remaining distance to the goal using Euclidean distance.
        x1, y1 = node.state
        x2, y2 = self.goal
        return int(distance.euclidean((x1, y1), (x2, y2)))

In [74]:
class FirefightingNode(Node):
    def __init__(self, state, parent=None, action=None, path_cost=0):
        super().__init__(state, parent, action, path_cost)
        self.depth = 0 if parent is None else parent.depth + 1

In [75]:
def create_firefighting_graph(width, height, obstacles, start, goal):
    # Create a grid-based graph representing the firefighting environment.
    graph = [[''] * height for _ in range(width)]

    for x, y in obstacles:
        graph[x][y] = 'Obstacle'

    graph[start[0]][start[1]] = 'Start'
    graph[goal[0]][goal[1]] = 'Goal'

    return graph

In [76]:
# Define the dimensions of the grid and the initial state.
start = (1, 1)
goal = (6, 7)

In [77]:
# Create the grid-based graph.
graph = create_firefighting_graph(width, height, obstacles, start, goal)

# Create a FirefightingProblem instance with the defined initial and goal states.
problem = FirefightingProblem(start, goal, graph)

# Find the optimal path using A* search.
#from scipy.spatial import distance
#goal_node = astar_search(problem, display=True)

# Find the goal node using best first graph search.
#goal_node = best_first_graph_search(problem, problem.h, display=True)

# Find the goal node using uniform cost search.
goal_node = uniform_cost_search(problem, problem.h)

# Extract the optimal path from the goal node.
optimal_path = [n.state for n in goal_node.path()]

73 paths have been expanded and 4 paths remain in the frontier


In [78]:
# Get the number of iterations
iterations = goal_node.path_cost

# Print the optimal path and other information.
print("Optimal Path:", optimal_path)
print("Number of Iterations:", iterations)
print(goal_node)

Optimal Path: [(1, 1), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7)]
Number of Iterations: 13
<Node (6, 7)>


In [79]:
def add_path(self, obstacles):
    for each in obstacles:
        print(each)
        x, y = each
        self.add_thing(Path(), (x, y))


add_path(forrest, optimal_path)

(1, 1)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(1, 7)
(2, 7)
(3, 7)
(4, 7)
(5, 7)
(6, 7)


In [80]:
forrest.run(0, delay=0.01)

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
